# Install & Import Dependencies

In [1]:
!pip install -q transformers datasets accelerate evaluate torch
!pip install -q transformers datasets accelerate evaluate torch contractions
!pip install -q nlpaug

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [23]:

import pandas as pd
import re
import unicodedata
import contractions
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from tqdm import tqdm
from google.colab import drive
from sklearn.model_selection import train_test_split, KFold
import nlpaug.augmenter.word as naw  # For data augmentation
from datasets import Dataset as HFDataset

# Add NLTK resource download
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
print("NLTK resources downloaded successfully")


NLTK resources downloaded successfully


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Conecting to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading Data & inspecting data

In [4]:
train_path = "/content/training_set_rel3.tsv"
valid_path = "/content/valid_set.tsv"
test_path = "/content/test_set.tsv"

# Loading TSV files with tab separator and ISO-8859-1 encoding for compatibility.
train_df = pd.read_csv(train_path, sep="\t", encoding="ISO-8859-1") # Tab-separated handles Western chars.
valid_df = pd.read_csv(valid_path, sep="\t", encoding="ISO-8859-1")
test_df = pd.read_csv(test_path, sep="\t", encoding="ISO-8859-1")
print("Datasets loaded successfully ")

Datasets loaded successfully 


In [5]:
print("Train Data Columns:", train_df.columns.tolist())
print("Valid Data Columns:", valid_df.columns.tolist())
print("Test Data Columns:", test_df.columns.tolist())

Train Data Columns: ['essay_id', 'essay_set', 'essay', 'rater1_domain1', 'rater2_domain1', 'rater3_domain1', 'domain1_score', 'rater1_domain2', 'rater2_domain2', 'domain2_score', 'rater1_trait1', 'rater1_trait2', 'rater1_trait3', 'rater1_trait4', 'rater1_trait5', 'rater1_trait6', 'rater2_trait1', 'rater2_trait2', 'rater2_trait3', 'rater2_trait4', 'rater2_trait5', 'rater2_trait6', 'rater3_trait1', 'rater3_trait2', 'rater3_trait3', 'rater3_trait4', 'rater3_trait5', 'rater3_trait6']
Valid Data Columns: ['essay_id', 'essay_set', 'essay', 'domain1_predictionid', 'domain2_predictionid']
Test Data Columns: ['essay_id', 'essay_set', 'essay', 'domain1_predictionid', 'domain2_predictionid']


In [6]:
print(valid_df.head())

   essay_id  essay_set                                              essay  \
0      1788          1  Dear @ORGANIZATION1, @CAPS1 more and more peop...   
1      1789          1  Dear @LOCATION1 Time @CAPS1 me tell you what I...   
2      1790          1  Dear Local newspaper, Have you been spending a...   
3      1791          1  Dear Readers, @CAPS1 you imagine how life woul...   
4      1792          1  Dear newspaper, I strongly believe that comput...   

   domain1_predictionid  domain2_predictionid  
0                  1788                   NaN  
1                  1789                   NaN  
2                  1790                   NaN  
3                  1791                   NaN  
4                  1792                   NaN  


# Select Relevant Features

In [11]:
train_df = train_df[['essay', 'domain1_score', 'essay_set']]

In [12]:
train_df.dropna(subset=['essay', 'domain1_score', 'essay_set'], inplace=True)
train_df.drop_duplicates(subset=['essay'], inplace=True)
train_df['domain1_score'] = train_df['domain1_score'].astype(float)

In [16]:
train_df['normalized_score'] = train_df.groupby('essay_set')['domain1_score'].transform(
    lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten()
)

In [24]:
# Definingg data augmentation function
def augment_text(text, aug_p=0.3):
    aug = naw.SynonymAug(aug_p=aug_p)  # Synonym replacement
    augmented_text = aug.augment(text)
    return augmented_text[0] if isinstance(augmented_text, list) else augmented_text

# Applying augmentation to training data
def augment_dataset(df, num_augmentations=1):
    augmented_data = []
    for _, row in df.iterrows():
        essay = row['essay']
        domain1_score = row['domain1_score']
        essay_set = row['essay_set']
        normalized_score = row['normalized_score']  # just ensuring normalized_score is included
        # Original sample
        augmented_data.append({
            'essay': essay,
            'domain1_score': domain1_score,
            'essay_set': essay_set,
            'normalized_score': normalized_score
        })
        # Augmented samples
        for _ in range(num_augmentations):
            augmented_essay = augment_text(essay)
            augmented_data.append({
                'essay': augmented_essay,
                'domain1_score': domain1_score,
                'essay_set': essay_set,
                'normalized_score': normalized_score
            })
    return pd.DataFrame(augmented_data)

# applying augmentation to train_df
print("Augmenting dataset...")
train_df = augment_dataset(train_df, num_augmentations=1)  # Double the dataset size
print("Augmented Train Data Shape:", train_df.shape)

Augmenting dataset...
Augmented Train Data Shape: (25944, 4)


In [22]:
import nltk

nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')  #its required for SynonymAug
print("NLTK resources downloaded successfully")

NLTK resources downloaded successfully


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
# Selectingg Relevant Features
train_df = train_df[['essay', 'domain1_score', 'essay_set']].copy()  # Creating a copy to avoid SettingWithCopyWarning

# Dropping rows with missing values
train_df = train_df.dropna(subset=['essay', 'domain1_score', 'essay_set'])

# Dropping duplicate essays
train_df = train_df.drop_duplicates(subset=['essay'])

# Ensuring target variable is in float format
train_df.loc[:, 'domain1_score'] = train_df['domain1_score'].astype(float)

# Normalizing scores per essay_set (0–1 range)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_df.loc[:, 'normalized_score'] = train_df.groupby('essay_set')['domain1_score'].transform(
    lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten()
)

In [27]:

def clean_text(text):
    text = text.lower()
    text = contractions.fix(text)  # Expanding contractions
    text = unicodedata.normalize("NFKD", text)
    text = re.sub(r"[^\w\s]", "", text)  # Removing punctuation and special characters
    text = re.sub(r"\s+", " ", text).strip()  # Removing extra whitespace
    return text


train_df = train_df[['essay', 'domain1_score', 'essay_set']].copy()

train_df = train_df.dropna(subset=['essay', 'domain1_score', 'essay_set'])

train_df = train_df.drop_duplicates(subset=['essay'])

# Ensuriing target variable is in float format
train_df.loc[:, 'domain1_score'] = train_df['domain1_score'].astype(float)

# Normalizing scores per essay_set (0–1 range)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_df.loc[:, 'normalized_score'] = train_df.groupby('essay_set')['domain1_score'].transform(
    lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten()
)

# Apply text cleaning (after augmentation to ensure consistency)
train_df.loc[:, 'essay'] = train_df['essay'].apply(clean_text)
print("Text Cleaning: Normalization & Formatting Done")

# Prepare for k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
folds = list(kf.split(train_df))
print("K-Fold Cross-Validation Prepared with 5 Folds")

Text Cleaning: Normalization & Formatting Done
K-Fold Cross-Validation Prepared with 5 Folds


# Text Cleaning Function

In [ ]:
def clean_text(text):
    text = text.lower()
    text = contractions.fix(text)  # Expanding contractions.
    text = unicodedata.normalize("NFKD", text)
    text = re.sub(r"[^\w\s]", "", text)  # Removing punctuation and special characters.
    text = re.sub(r"\s+", " ", text).strip()  # Removing extra whitespace.
    return text

train_data['essay'] = train_data['essay'].apply(clean_text)
valid_data['essay'] = valid_data['essay'].apply(clean_text)
test_df['essay'] = test_df['essay'].apply(clean_text)
print("Text Cleaning: Normalization & Formatting Done")

Text Cleaning: Normalization & Formatting Done


# Tokenization & Truncation

In [30]:
# Apply augmentation to train_df
print("Augmenting dataset...")
train_df = augment_dataset(train_df, num_augmentations=0)  # Skipping augmentation
print("Train Data Shape (No Augmentation):", train_df.shape)

Augmenting dataset...
Train Data Shape (No Augmentation): (25944, 4)


**I have forgot to save the results of this cell and overwrite it by the cell after, however I had screenshot for its results and mentiond it in the report**

In [ ]:
# Initialize tokenizer
model_name = "microsoft/deberta-v3-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/deberta-aes",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    learning_rate=3e-5,  # Adjusted learning rate
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,  # Increase epochs for better convergence
    weight_decay=0.01,
    report_to="none",
    fp16=True,
    logging_strategy="epoch"
)

# Define metrics for evaluation
import numpy as np
from sklearn.metrics import cohen_kappa_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.squeeze(predictions)
    labels = labels.astype(float)
    mse = np.mean((predictions - labels) ** 2)
    mae = np.mean(np.abs(predictions - labels))
    predictions_rounded = np.round(predictions * 10).astype(int)  # Scale 0–1 to 0–10
    labels_rounded = np.round(labels * 10).astype(int)
    qwk_score = cohen_kappa_score(labels_rounded, predictions_rounded, weights="quadratic")
    return {
        "mse": mse,
        "mae": mae,
        "qwk": qwk_score
    }

# K-Fold Cross-Validation Training Loop
fold_results = []
for fold, (train_idx, val_idx) in enumerate(folds):
    print(f"\nTraining Fold {fold + 1}/{len(folds)}")

    # Split data for this fold
    train_data = train_df.iloc[train_idx]
    valid_data = train_df.iloc[val_idx]

    # Create datasets using normalized_score
    train_dataset = EssayDataset(train_data["essay"].tolist(), train_data["normalized_score"].tolist())
    valid_dataset = EssayDataset(valid_data["essay"].tolist(), valid_data["normalized_score"].tolist())

    # Convert to Hugging Face Dataset format
    train_hf_dataset = HFDataset.from_dict({
        "input_ids": train_dataset.encodings["input_ids"].tolist(),
        "attention_mask": train_dataset.encodings["attention_mask"].tolist(),
        "labels": train_dataset.labels.tolist()
    })
    valid_hf_dataset = HFDataset.from_dict({
        "input_ids": valid_dataset.encodings["input_ids"].tolist(),
        "attention_mask": valid_dataset.encodings["attention_mask"].tolist(),
        "labels": valid_dataset.labels.tolist()
    })

    # Reset model for each fold
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_hf_dataset,
        eval_dataset=valid_hf_dataset,
        compute_metrics=compute_metrics
    )

    # Train
    trainer.train()

    # Evaluate
    eval_results = trainer.evaluate()
    print(f"Fold {fold + 1} Evaluation Results:", eval_results)
    fold_results.append(eval_results)

    # Check sample predictions (fix the slicing issue)
    valid_subset = valid_hf_dataset.select(range(min(10, len(valid_hf_dataset))))
    predictions = trainer.predict(valid_subset).predictions
    print(f"Fold {fold + 1} Sample Predictions:", predictions.flatten())

# Average results across folds
avg_results = {
    "avg_mse": np.mean([r["eval_mse"] for r in fold_results]),
    "avg_mae": np.mean([r["eval_mae"] for r in fold_results]),
    "avg_qwk": np.mean([r["eval_qwk"] for r in fold_results])
}
print("\nAverage Results Across Folds:", avg_results)

**this one gives close results, after adjusting the Lr and minor things like the weight_decay for handling overfitting and so on**

**tried to train only one fold for both cells due to the significant power (2hrs for the 3epchos on A100)**

In [31]:
# Initialize tokenizer
model_name = "microsoft/deberta-v3-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/deberta-aes",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    learning_rate=3e-5,  # Adjusted learning rate
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,  # Increase epochs for better convergence
    weight_decay=0.01,
    report_to="none",
    fp16=True,
    logging_strategy="epoch"
)

# Define metrics for evaluation
import numpy as np
from sklearn.metrics import cohen_kappa_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.squeeze(predictions)
    labels = labels.astype(float)
    mse = np.mean((predictions - labels) ** 2)
    mae = np.mean(np.abs(predictions - labels))
    predictions_rounded = np.round(predictions * 10).astype(int)  # Scale 0–1 to 0–10
    labels_rounded = np.round(labels * 10).astype(int)
    qwk_score = cohen_kappa_score(labels_rounded, predictions_rounded, weights="quadratic")
    return {
        "mse": mse,
        "mae": mae,
        "qwk": qwk_score
    }

# K-Fold Cross-Validation Training Loop
fold_results = []
for fold, (train_idx, val_idx) in enumerate(folds):
    print(f"\nTraining Fold {fold + 1}/{len(folds)}")

    # Split data for this fold
    train_data = train_df.iloc[train_idx]
    valid_data = train_df.iloc[val_idx]

    # Create datasets using normalized_score
    train_dataset = EssayDataset(train_data["essay"].tolist(), train_data["normalized_score"].tolist())
    valid_dataset = EssayDataset(valid_data["essay"].tolist(), valid_data["normalized_score"].tolist())

    # Convert to Hugging Face Dataset format
    train_hf_dataset = HFDataset.from_dict({
        "input_ids": train_dataset.encodings["input_ids"].tolist(),
        "attention_mask": train_dataset.encodings["attention_mask"].tolist(),
        "labels": train_dataset.labels.tolist()
    })
    valid_hf_dataset = HFDataset.from_dict({
        "input_ids": valid_dataset.encodings["input_ids"].tolist(),
        "attention_mask": valid_dataset.encodings["attention_mask"].tolist(),
        "labels": valid_dataset.labels.tolist()
    })

    # Reset model for each fold
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_hf_dataset,
        eval_dataset=valid_hf_dataset,
        compute_metrics=compute_metrics
    )

    # Train
    trainer.train()

    # Evaluate
    eval_results = trainer.evaluate()
    print(f"Fold {fold + 1} Evaluation Results:", eval_results)
    fold_results.append(eval_results)

    # Check sample predictions (fix the slicing issue)
    valid_subset = valid_hf_dataset.select(range(min(10, len(valid_hf_dataset))))
    predictions = trainer.predict(valid_subset).predictions
    print(f"Fold {fold + 1} Sample Predictions:", predictions.flatten())

# Average results across folds
avg_results = {
    "avg_mse": np.mean([r["eval_mse"] for r in fold_results]),
    "avg_mae": np.mean([r["eval_mae"] for r in fold_results]),
    "avg_qwk": np.mean([r["eval_qwk"] for r in fold_results])
}
print("\nAverage Results Across Folds:", avg_results)

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(



Training Fold 1/5


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Mse,Mae,Qwk
1,0.055100,0.065596,0.065596,0.200046,0.000000
2,0.041500,0.058801,0.058801,0.190596,0.000000
3,0.038900,0.057381,0.057381,0.188589,0.000000


Sample Predictions (first 10): [0.68310547 0.68310547 0.68310547 0.68310547 0.68310547 0.68310547
 0.68310547 0.68310547 0.68310547 0.68310547]
Sample Labels (first 10): [0.69999999 0.40000001 1.         0.60000002 0.60000002 0.80000001
 0.89999998 0.2        0.69999999 0.69999999]
Rounded Predictions (first 10): [7 7 7 7 7 7 7 7 7 7]
Rounded Labels (first 10): [ 7  4 10  6  6  8  9  2  7  7]
Sample Predictions (first 10): [0.6328125 0.6328125 0.6328125 0.6328125 0.6328125 0.6328125 0.6328125
 0.6328125 0.6328125 0.6328125]
Sample Labels (first 10): [0.69999999 0.40000001 1.         0.60000002 0.60000002 0.80000001
 0.89999998 0.2        0.69999999 0.69999999]
Rounded Predictions (first 10): [6 6 6 6 6 6 6 6 6 6]
Rounded Labels (first 10): [ 7  4 10  6  6  8  9  2  7  7]
Sample Predictions (first 10): [0.6098633 0.6098633 0.6098633 0.6098633 0.6098633 0.6098633 0.6098633
 0.6098633 0.6098633 0.6098633]
Sample Labels (first 10): [0.69999999 0.40000001 1.         0.60000002 0.60000002 0.

Sample Predictions (first 10): [0.6098633 0.6098633 0.6098633 0.6098633 0.6098633 0.6098633 0.6098633
 0.6098633 0.6098633 0.6098633]
Sample Labels (first 10): [0.69999999 0.40000001 1.         0.60000002 0.60000002 0.80000001
 0.89999998 0.2        0.69999999 0.69999999]
Rounded Predictions (first 10): [6 6 6 6 6 6 6 6 6 6]
Rounded Labels (first 10): [ 7  4 10  6  6  8  9  2  7  7]
Fold 1 Evaluation Results: {'eval_loss': 0.05738134682178497, 'eval_mse': 0.057381345178923286, 'eval_mae': 0.18858916465229647, 'eval_qwk': 0.0, 'eval_runtime': 82.7571, 'eval_samples_per_second': 62.702, 'eval_steps_per_second': 15.684, 'epoch': 3.0}
Sample Predictions (first 10): [0.6098633 0.6098633 0.6098633 0.6098633 0.6098633 0.6098633 0.6098633
 0.6098633 0.6098633 0.6098633]
Sample Labels (first 10): [0.69999999 0.40000001 1.         0.60000002 0.60000002 0.80000001
 0.89999998 0.2        0.69999999 0.69999999]
Rounded Predictions (first 10): [6 6 6 6 6 6 6 6 6 6]
Rounded Labels (first 10): [ 7  4 

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Mse,Mae,Qwk
1,0.054600,0.105663,0.105663,0.266249,0.000000


Sample Predictions (first 10): [0.8120117 0.8120117 0.8120117 0.8120117 0.8120117 0.8120117 0.8120117
 0.8120117 0.8120117 0.8120117]
Sample Labels (first 10): [0.60000002 0.69999999 0.80000001 0.69999999 1.         0.40000001
 0.60000002 0.1        0.89999998 0.60000002]
Rounded Predictions (first 10): [8 8 8 8 8 8 8 8 8 8]
Rounded Labels (first 10): [ 6  7  8  7 10  4  6  1  9  6]


KeyboardInterrupt: 

# Loading Model

# Post-Training Evaluation

In [ ]:
# Convert EssayDataset to a format Trainer expects
from datasets import Dataset as HFDataset

train_hf_dataset = HFDataset.from_dict({
    "input_ids": train_dataset.encodings["input_ids"].tolist(),
    "attention_mask": train_dataset.encodings["attention_mask"].tolist(),
    "labels": train_dataset.labels.tolist()
})
valid_hf_dataset = HFDataset.from_dict({
    "input_ids": valid_dataset.encodings["input_ids"].tolist(),
    "attention_mask": valid_dataset.encodings["attention_mask"].tolist(),
    "labels": valid_dataset.labels.tolist()
})

# Single run
batch_size = 4
learning_rate = 3e-5

print(f"Training with Batch Size: {batch_size}, Learning Rate: {learning_rate}")
training_args.per_device_train_batch_size = batch_size
training_args.per_device_eval_batch_size = batch_size
training_args.learning_rate = learning_rate

# Reset model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
trainer.model = model

# Update trainer datasets
trainer.train_dataset = train_hf_dataset
trainer.eval_dataset = valid_hf_dataset

# Train
trainer.train()

# Check predictions to debug constant eval loss
predictions = trainer.predict(valid_hf_dataset[:10]).predictions
print("Sample Predictions after 1 epoch:", predictions.flatten())

Training with Batch Size: 4, Learning Rate: 3e-05


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Model Preparation Time
1,0.258700,0.079961,0.441300


KeyError: 0